In [7]:
#2019.10.18 准备写一个大盘分析程序，将相关数据显示在网页上，然后后台定时运行，分析每天结果，我想看时打开网页就行。
import tushare as ts
pro = ts.pro_api() 
#初始化
df1 = pro.index_dailybasic(start_date='20191008',end_date='20191017',ts_code='000001.SH',fields='ts_code,trade_date,total_mv,turnover_rate')
#index_dailybasic提供大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df2 = pro.index_daily(ts_code='000001.SH',start_date='20191008',end_date='20191017')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df2['total_mv']=df1.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
#--------------------我是分割线------------------------------------------------------------
#2019.10.19
#原本想将运行结果保存到数据库，然后用PHP读出来显示到网页上，今天突然想到PYTHON能不能直接保存结果为HTML？百度一下，居然发现DATAFRAME可以
#直接存为HTML文件，这下省事了。
df2.to_html('files.html',escape=False,index=False,sparsify=True,border=1,index_names=True,header=True)

